In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "mistralai"

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir="/mistralai/", torch_dtype=torch.float16).to(
        "cuda")
tok = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir="/mistralai/", torch_dtype=torch.float16)

for param in model.parameters():
    param.requires_grad = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [18]:
device = "cuda"

In [4]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

# load model and tokenizer
model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-Mistral', cache_dir="/SFR_Mistral/", torch_dtype=torch.float16).to(device)

# no special token
tok = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral', cache_dir="/SFR_Mistral/", torch_dtype=torch.float16, add_special_tokens=False) 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
for param in model.parameters():
    param.requires_grad = False

In [19]:
tok.add_special_tokens({'pad_token': tok.eos_token})

0

In [20]:
def get_model_layer_attentions(sentence, token_ids, model, tokenizer, layer):
    input_ids = tokenizer(sentence, padding='max_length', max_length=100, return_tensors="pt")["input_ids"].to(device)
    lst = token_ids + input_ids.tolist()[0]
    input_ids = torch.tensor([lst], dtype=torch.int).to(device)
    with torch.no_grad():
        output = model(input_ids, output_attentions=True)
    
    flattened_attentions = [attn_layer.flatten() for attn_layer in output.attentions]
    flattened_vector = torch.cat(flattened_attentions)  # This concatenates them into a single vector
    return flattened_vector

def get_model_layer_hidden_states(sentence, token_ids, model, tokenizer, layer):
    input_ids = tokenizer(sentence, padding='max_length', max_length=100, return_tensors="pt")["input_ids"].to(device)
    lst = token_ids + input_ids.tolist()[0]
    input_ids = torch.tensor([lst], dtype=torch.int).to(device)
    with torch.no_grad():
        output = model(input_ids, output_hidden_states=True, output_attentions=True)
    flattened_hidden_states = [attn_layer.flatten() for attn_layer in output.hidden_states[-1:]]
    flattened_vector = torch.cat(flattened_hidden_states)  # This concatenates them into a single vector
    return flattened_vector

In [21]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

In [89]:
import pickle
with open('SFR_Mistral_IMDB_1.pkl', 'rb') as f:
    json_str_1 = pickle.load(f)

json_str_1.keys()

dict_keys([507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615])

In [90]:
import pickle
with open('SFR_Mistral_IMDB_2.pkl', 'rb') as f:
    json_str_2 = pickle.load(f)

json_str_2.keys()

dict_keys([400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506])

In [91]:
import pickle
with open('SFR_Mistral_IMDB_3.pkl', 'rb') as f:
    json_str_3 = pickle.load(f)

json_str_3.keys()

dict_keys([267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373])

In [92]:
import pickle
with open('SFR_Mistral_IMDB_4.pkl', 'rb') as f:
    json_str_4 = pickle.load(f)

json_str_4.keys()

dict_keys([208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266])

In [93]:
import pickle
with open('SFR_Mistral_IMDB_5.pkl', 'rb') as f:
    json_str_5 = pickle.load(f)

json_str_5.keys()

dict_keys([180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207])

In [94]:
import pickle
with open('SFR_Mistral_IMDB_6.pkl', 'rb') as f:
    json_str_6 = pickle.load(f)

json_str_6.keys()

dict_keys([110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165])

In [95]:
import pickle
with open('SFR_Mistral_IMDB_7.pkl', 'rb') as f:
    json_str_7 = pickle.load(f)

json_str_7.keys()

dict_keys([54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110])

In [96]:
import pickle
with open('SFR_Mistral_IMDB_8.pkl', 'rb') as f:
    json_str_8 = pickle.load(f)

json_str_8.keys()

dict_keys([27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])

In [97]:
import pickle
with open('SFR_Mistral_IMDB_9.pkl', 'rb') as f:
    json_str_9 = pickle.load(f)

json_str_9.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])

In [98]:
import pickle
with open('SFR_Mistral_IMDB_10.pkl', 'rb') as f:
    json_str_10 = pickle.load(f)

json_str_10.keys()

dict_keys([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53])

# Count perturbation performance

In [111]:
hidden_states_pos = []
layer = -1
counter_success = 0
counter = 0
total = 0
for json_str in [json_str_4, json_str_5, json_str_6, json_str_7, json_str_8, json_str_9, json_str_10]:
    for index in json_str:
        if 'is_success' not in json_str[index].keys():
            continue
        if json_str[index]['is_success'] > 0:
            counter += 1
            if json_str[index]['is_success'] > 1:
                counter_success += 1
            prompt_text = json_str[index]["prompt_text"]
            token_ids = json_str[index]["token_ids"]
            
        total += 1
len(hidden_states_pos), counter / total, counter_success / total

(0, 0.2247191011235955, 0.2247191011235955)

# Detect the GGPP prefix

In [99]:
json_str_filter = {}
j = 0
for json_str in [json_str_1, json_str_2, json_str_3, json_str_4, json_str_5, json_str_6, json_str_7, json_str_8, json_str_9, json_str_10]:
    for i in json_str:
        if j >= 100:
            break
        if json_str[i]['is_success'] > 1:
            json_str_filter[i] = json_str[i]
            j += 1
j

100

In [134]:
import csv
from struct import pack, unpack
import lzma
# import blosc
import numpy as np
import pickle
import torch

import csv

tconst_title = {}
count = 0
with open("/title.basics.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        if count > 0:
            
            tconst_title[row[0]] = row[2]
            
        count += 1

device = torch.device("cuda")

In [135]:
prompts = []
name_tt = {}
nconst_name = []
nconst_profession = []
nconst_embedding = []

with open("/datasets/name.basics.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    counter = 0
    
    for row in rd:
        if counter > 1000:           
            break
        if counter > 0:            
            prompt = row[1] +" was born in "+row[2]+", "+"and died in "+row[3]+". He/She's primary professions are "+', '.join(map(str, row[4].split(",")))+"."
            
            tts = row[5].split(",")
            prompt += " He/She is known for movies:"
            for t in tts:
                if t == tts[-1]:
                    prompt += " '" + tconst_title[t]+"'."
                else:
                    prompt += " '" + tconst_title[t]+"',"

            max_length = 526
            input_texts = prompt
            batch_dict = tok(input_texts, max_length=max_length, padding=True, truncation=True, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = model(**batch_dict)
                embedding = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask']).to("cpu")
        
            nconst_embedding.append(embedding[0])
            prompts.append(prompt)

        counter += 1
        

In [136]:
from DGD.help_functions import get_embedding, perturb_sentence, rank_tokens_by_importance, get_initial_ids, user_prompt_generation, check_success_origin, check_MSEloss, get_first_output_token, get_neighbor_ids, check_success_target_origin, get_relevant_documents
from DGD.DGD import get_optimized_prefix_embedding
import random

# Set dataset
dataset = "imdb"
query_mode = "for_objects"

counter = 0

hidden_states_positive = []
layer = -1
origin_token_list = {}
for index in json_str_filter:

    user_prompt = json_str_filter[index]["user_prompt"]
    initial_firstk = 10
    initial_ids = [random.randint(0, 32000) for _ in range(initial_firstk)]
    user_prompt_id = index
    target_prompt_id = index
    
    topk_result = 10
    top_k = get_relevant_documents(torch.stack(nconst_embedding), user_prompt, topk_result, model, tok)
    is_success = check_success_origin(target_prompt_id, user_prompt_id, top_k)
    if is_success <= 0:
        counter += 1
        
    origin_token_list[index] = initial_ids
len(origin_token_list.keys()), counter

task_query:  According to IMDB dataset and your knowledges, what movies Terrence Malick has worked on and what were her/his roles?
top_k:  [516 653 413 765 415  75 264 646 179 973]
task_query:  According to IMDB dataset and your knowledges, what movies Michael Mann has worked on and what were her/his roles?
top_k:  [519 340 435 879 839 322 139 608 775 298]
task_query:  According to IMDB dataset and your knowledges, what movies Colm Meaney has worked on and what were her/his roles?
top_k:  [537 558 341 637 392 538 982 295 545 535]
task_query:  According to IMDB dataset and your knowledges, what movies Roger Moore has worked on and what were her/his roles?
top_k:  [548 210 124 725 616 960 323  56 192 934]
task_query:  According to IMDB dataset and your knowledges, what movies Judd Nelson has worked on and what were her/his roles?
top_k:  [554 686 545 207  22 130 882 638 664 362]
task_query:  According to IMDB dataset and your knowledges, what movies Leonard Nimoy has worked on and what w

(100, 0)

In [137]:
counter

0

In [142]:
from DGD.help_functions import get_embedding, perturb_sentence, rank_tokens_by_importance, get_initial_ids, user_prompt_generation, check_success, check_MSEloss, get_first_output_token, get_neighbor_ids
from DGD.DGD import get_optimized_prefix_embedding

# Set dataset
dataset = "imdb"
query_mode = "for_objects"

hidden_states_negative = []
layer = -1
for index in json_str_filter:
    prompt_text = json_str_filter[index]["prompt_text"]
    token_ids = json_str_filter[index]["token_ids"]
    object = ""
    sentence = user_prompt_generation(prompt_text, object, dataset, query_mode)
    print(sentence)
    print(token_ids)
    hidden_state = get_model_layer_hidden_states(sentence, token_ids, model, tok, layer)
    hidden_states_negative.append(hidden_state.to("cpu"))
len(hidden_states_negative)

According to IMDB dataset and your knowledges, what movies Terrence Malick has worked on and what were her/his roles?
[21511, 393, 1375, 403, 5381, 1401, 25346, 14800, 13127, 3999]
According to IMDB dataset and your knowledges, what movies Michael Mann has worked on and what were her/his roles?
[17332, 24158, 403, 7043, 3616, 12567, 28740, 21309, 24978, 28784]
According to IMDB dataset and your knowledges, what movies Colm Meaney has worked on and what were her/his roles?
[20991, 27542, 263, 4285, 17300, 4764, 6283, 18526, 31739, 5202]
According to IMDB dataset and your knowledges, what movies Roger Moore has worked on and what were her/his roles?
[21511, 1739, 973, 403, 5381, 1259, 7826, 6926, 20215, 23262]
According to IMDB dataset and your knowledges, what movies Judd Nelson has worked on and what were her/his roles?
[393, 15536, 4262, 1896, 16889, 403, 2236, 27191, 24694, 22193]
According to IMDB dataset and your knowledges, what movies Leonard Nimoy has worked on and what were her

100

In [143]:
from DGD.help_functions import get_embedding, perturb_sentence, rank_tokens_by_importance, get_initial_ids, user_prompt_generation, check_success, check_MSEloss, get_first_output_token, get_neighbor_ids
from DGD.DGD import get_optimized_prefix_embedding

# Set dataset
dataset = "imdb"
query_mode = "for_objects"

hidden_states_positive = []
layer = -1
counter = 0
for index in json_str_filter:
    # if counter == 100:
    #     break
    prompt_text = json_str_filter[index]["prompt_text"]
    initial_ids = origin_token_list[index]
    object = ""
    sentence = user_prompt_generation(prompt_text, object, dataset, query_mode)
    print(sentence)
    print(initial_ids)
    hidden_state = get_model_layer_hidden_states(sentence, initial_ids, model, tok, layer)
    hidden_states_positive.append(hidden_state.to("cpu"))
    counter += 1
len(hidden_states_positive)

According to IMDB dataset and your knowledges, what movies Terrence Malick has worked on and what were her/his roles?
[9376, 4889, 26013, 28444, 8040, 26770, 24555, 19725, 8141, 20266]
According to IMDB dataset and your knowledges, what movies Michael Mann has worked on and what were her/his roles?
[28164, 12506, 2075, 9593, 21554, 24638, 1185, 15282, 20487, 31520]
According to IMDB dataset and your knowledges, what movies Colm Meaney has worked on and what were her/his roles?
[10151, 4283, 6794, 17052, 15596, 8603, 2362, 28164, 21659, 24509]
According to IMDB dataset and your knowledges, what movies Roger Moore has worked on and what were her/his roles?
[8751, 12707, 25467, 8239, 268, 6266, 5262, 2995, 3180, 19574]
According to IMDB dataset and your knowledges, what movies Judd Nelson has worked on and what were her/his roles?
[18029, 31205, 2527, 7836, 8680, 17662, 24113, 9950, 27361, 22216]
According to IMDB dataset and your knowledges, what movies Leonard Nimoy has worked on and wh

100

In [144]:
X_1 = []
y_1 = []
X_2 = []
y_2 = []
counter = 0
size = hidden_states_negative[0].shape[0]
for matrix in hidden_states_positive:
    if counter == 100:
        break
    if matrix.shape[0] != size:
        continue
    print(matrix.shape)
    X_1.append(matrix.to("cpu"))
    y_1.append(0)
    
    counter += 1
    
counter = 0
for matrix in hidden_states_negative:
    if counter == 100:
        break
    if matrix.shape[0] != size:
        continue
    print(matrix.shape)
    X_2.append(matrix.to("cpu"))
    y_2.append(1)

    counter += 1

torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([450560])
torch.Size([4

In [145]:
len(X_1), len(X_2)

(100, 100)

In [146]:
# Train:test = 6:4, hidden_states, imdb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, accuracy_score
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
import time

auroc_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
accuracy_lst = []
training_times = []
response_times = []
for seed in range(0, 10):
    shuffled_X_1, shuffled_X_2, shuffled_y_1, shuffled_y_2 = split_four_lists_randomly(X_1, X_2, y_1, y_2, seed)
    X_train = shuffled_X_1[:60] + shuffled_X_2[:60]
    X_test = shuffled_X_1[60:] + shuffled_X_2[60:]
    y_train = shuffled_y_1[:60] + shuffled_y_2[:60]
    y_test = shuffled_y_1[60:] + shuffled_y_2[60:]

    X_train, y_train = split_lists_randomly(X_train, y_train, seed)
    X_test, y_test = split_lists_randomly(X_test, y_test, seed)
    print(len(X_train), len(y_train), len(X_test), len(y_test))
    clf = LogisticRegression()
    
    # Start time
    start_time = time.perf_counter()
    
    clf.fit(X_train, y_train)
    
    # End time
    end_time = time.perf_counter()

    # Calculate response time
    training_time = end_time - start_time
    training_times.append(training_time)

    print("training_time: ", training_time)

    # Start time
    start_time = time.perf_counter()
    
    y_pred = clf.predict(X_test)
    
    # End time
    end_time = time.perf_counter()
    
    # Calculate response time
    response_time = end_time - start_time
    response_times.append(response_time)

    print("response_time: ", response_time)
    
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

        
    auroc_lst.append(auroc)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    accuracy_lst.append(accuracy)
    print(str(seed) + " :")
    print(auroc)
    print(" ")
sum(auroc_lst) / len(auroc_lst), sum(f1_lst) / len(f1_lst), sum(recall_lst) / len(recall_lst), sum(precision_lst) / len(precision_lst), sum(accuracy_lst) / len(accuracy_lst)

120 120 80 80
training_time:  4.46355907805264
response_time:  0.20115510933101177
0 :
1.0
 
120 120 80 80
training_time:  4.4981503849849105
response_time:  0.20134748984128237
1 :
1.0
 
120 120 80 80
training_time:  5.7243487681262195
response_time:  0.20093853818252683
2 :
1.0
 
120 120 80 80
training_time:  6.029270411003381
response_time:  0.20127836661413312
3 :
1.0
 
120 120 80 80
training_time:  3.6956862648949027
response_time:  0.20144648803398013
4 :
1.0
 
120 120 80 80
training_time:  5.4898805767297745
response_time:  0.22186102718114853
5 :
1.0
 
120 120 80 80
training_time:  4.394320995081216
response_time:  0.2013642732053995
6 :
1.0
 
120 120 80 80
training_time:  4.403729849029332
response_time:  0.2185397632420063
7 :
1.0
 
120 120 80 80
training_time:  3.665802648290992
response_time:  0.20120144123211503
8 :
1.0
 
120 120 80 80
training_time:  4.561057776212692
response_time:  0.2010862729512155
9 :
1.0
 


(1.0, 0.9936708860759493, 0.9875, 1.0, 0.99375)

In [ ]:
# Train:test = 6:4, attentions, imdb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, accuracy_score
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
import time

auroc_lst = []
f1_lst = []
recall_lst = []
precision_lst = []
accuracy_lst = []
training_times = []
response_times = []
for seed in range(0, 10):
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=seed)
    shuffled_X_1, shuffled_X_2, shuffled_y_1, shuffled_y_2 = split_four_lists_randomly(X_1, X_2, y_1, y_2, seed)
    X_train = shuffled_X_1[:60] + shuffled_X_2[:60]
    X_test = shuffled_X_1[60:] + shuffled_X_2[60:]
    y_train = shuffled_y_1[:60] + shuffled_y_2[:60]
    y_test = shuffled_y_1[60:] + shuffled_y_2[60:]

    X_train, y_train = split_lists_randomly(X_train, y_train, seed)
    X_test, y_test = split_lists_randomly(X_test, y_test, seed)
    # clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(500, 10), random_state=1)
    print(len(X_train), len(y_train), len(X_test), len(y_test))
    clf = LogisticRegression()
    
    # Start time
    start_time = time.perf_counter()
    
    clf.fit(X_train, y_train)
    
    # End time
    end_time = time.perf_counter()

    # Calculate response time
    training_time = end_time - start_time
    training_times.append(training_time)

    print("training_time: ", training_time)

    # Start time
    start_time = time.perf_counter()
    
    y_pred = clf.predict(X_test)
    
    # End time
    end_time = time.perf_counter()
    
    # Calculate response time
    response_time = end_time - start_time
    response_times.append(response_time)

    print("response_time: ", response_time)
    
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

        
    auroc_lst.append(auroc)
    f1_lst.append(f1)
    recall_lst.append(recall)
    precision_lst.append(precision)
    accuracy_lst.append(accuracy)
    print(str(seed) + " :")
    print(auroc)
    print(" ")
sum(auroc_lst) / len(auroc_lst), sum(f1_lst) / len(f1_lst), sum(recall_lst) / len(recall_lst), sum(precision_lst) / len(precision_lst), sum(accuracy_lst) / len(accuracy_lst)